# Prompt Engineering: Use OpenAI to Analyze Twitter Data 
This is a simple tutorial teaching prompt engineering basics and analyzing Twitter data with OpenAI large language models (LLM).
Please purchase an [OpenAI API](https://openai.com/index/openai-api/) and store it in a safe place. This tutorial uses [AWS Secretes Manager](https://aws.amazon.com/secrets-manager/) to store the API keys.  

## Large Language Model Basics
LLM repeatable predicts the next world using supervised learning. To predict the following sentence: 

`Learning data science in the cloud with AI`

A model needs to learn to predict the following steps:

|Input|Output|
|:---|---|
|Learning data science |in |
|Learning data science in |the | 
|Learning data science in the |cloud |
|Learning data science in the cloud |with |
|Learning data science in the cloud with |AI|

To train an LLM model:
1. Training a base LLM model on a large amount of training data to predict the next word 
2. Fine-tune on examples where outputs follow instructions in the input 
3. Human rates quality of different LLM outputs 
4. Tune LLM to generate outputs with higher rates using RLHF (Reinforcement learning from human feedback)

## Set up OpenAI Models

Load the API keys with AWS Secrets Manage Function 

In [1]:
import boto3
from botocore.exceptions import ClientError
import json

def get_secret(secret_name):
    region_name = "us-east-1"

    # Create a Secrets Manager client
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )

    try:
        get_secret_value_response = client.get_secret_value(
            SecretId=secret_name
        )
    except ClientError as e:
        raise e

    secret = get_secret_value_response['SecretString']
    
    return json.loads(secret)

## Install Python libraries.

- pymongo: manage the MongoDB database
- openai: call the OpenAI APIs.

In [2]:
pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 599.1/599.1 kB 21.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install pymongo

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 64.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


Load the OpenAI API key and define a `openai_help` function.

In [4]:
from openai import OpenAI

openai_api_key  = get_secret('openai')['api_key']
client = OpenAI(api_key=openai_api_key)
model = 'gpt-4o'
temperature = 0

def openai_help(messages, model=model, temperature =temperature ):
    messages = messages
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature

    )
    return response.choices[0].message.content

Temperature: 
- Low temperature: always choose the most likely response, reliable, predictable responses  
- High temperature: diverse responses, more creative responses

Tokens and Models: 
- LLM predicts tokens, which are commonly occurring sequences of characters. 
- One token is about four characters in English, and 100 tokens are roughly 75 words. Check [token estimate](https://platform.openai.com/tokenizer).
- Different models can process various amounts of tokens at different performance levels and costs. Check [OpenAI models](https://platform.openai.com/docs/models) for more details.

Roles:
- system: specify the overall tone or behavior of the assistant 
- user: instruction given to the LLM
- assistant: LLM responded content, we also can provide content in few-shot promoting or histories of conversations


A simple example using [gtp-4o](https://platform.openai.com/docs/models/gpt-4o) and temperature 0.

In [5]:
messages = [{"role": "user", "content": "What is the capital of USA"}]

print(openai_help(messages))

The capital of the United States is Washington, D.C.


Add a system message asking LLM to act as a high school teacher with different temperatures.

In [6]:
messages = [
    {"role": "system", "content": "use tone as a high school teacher"},
    {"role": "user", "content": "What is the capital of USA"}
    ]

print(openai_help(messages, temperature = 0.8))

The capital of the United States is Washington, D.C. It's an important city where you'll find the White House, the Capitol Building, and many other significant government institutions. If you have any more questions or need further information, feel free to ask!


Add assistant messages to teach LLM what `##` is.

In [7]:
messages = [
    {"role": "user", "content": "What is 1##1"},
    {"role": "assistant", "content": "it is 11"},
    {"role": "user", "content": "What is 2##2"},
    {"role": "assistant", "content": "it is 22"},
    {"role": "user", "content": "What is 3##3"},
    ]
print(openai_help(messages))

It is 33.


## Prompt Engineering Principles 
- Use delimiters to separate different parts of a prompt to provide clear instructions and prevent prompt injections.
- Structure outputs in JSON documents or other formats to use the outputs in subsequent steps 
- Few-shot promoting: provide successful examples of a task and then ask the model to perform a similar task. 
- Chain of thought reasoning: request a series of reasoning steps in prompts to help the model achieve correct answers
- Chain of prompts: split a task into multiple prompts where each prompt can focus on a sub-task at a time and take different actions at different stages. It saves tokens, is easier to test, can involve human input, or use external tools.
- Interactive process 
  1. Try something first 
  2. Analyses the result, identify errors, and redefine the prompt 
  3. Test the prompts with different datasets 


An example using delimiters, structured output and few-shot promoting:

In [8]:
delimiter = '###'
sentence1 = 'I love cat.'
sentence2 = 'I love dog.'
messages = [
    {"role": "system", "content": f"""analyze the sentiment in a sentence delimitered by {delimiter},
                                     return the result as a JSON document"""},
    {"role": "user", "content": f"{delimiter}{sentence1}{delimiter}"},
    {"role": "assistant", "content": "{sentiment:positive}"},
    {"role": "user", "content": f"{delimiter}{sentence2}{delimiter}"}
    ]

print(openai_help(messages))

{ "sentiment": "positive" }


## Analyze Twitter data

### Connect to the MongoDB cluster

In [9]:
import pymongo
from pymongo import MongoClient
mongodb_connect = get_secret('mongodb')['connection_string']

mongo_client = MongoClient(mongodb_connect)
db = mongo_client.demo # use or create a database named demo
tweet_collection = db.tweet_collection #use or create a collection named tweet_collection
tweet_collection.create_index([("tweet.id", pymongo.ASCENDING)],unique = True) # make sure the collected tweets are unique

'tweet.id_1'

### Extract Tweets

In [10]:
filter={

    
}
project={
    'tweet.text': 1, 
    'tweet.id': 1
}
#rename the client to mongo_client
result = mongo_client['demo']['tweet_collection'].find(
  filter=filter,
  projection=project
)

In [11]:
tweet_data = []
for tweet in result:
    tweet_data.append(tweet['tweet']['text'])


print(tweet_data)

['How’s your IT posture?\n(Hint: If your back’s straight but your firewall’s weak… we’ve got a problem.)\n\nCyber threats don’t slouch, they strike!\n\n#RVA #RichmondIT https://t.co/1bfArg96gQ', 'Cybersecurity \n\nIn today’s digital world, Cybersecurity is more important than ever. As technology advances, so do cyber threats. From data breaches to ransomware attacks, protecting sensitive information is critical.', "RT @TrendMicro: Trend Cybertron: the industry's first proactive cybersecurity AI. Our new AI agent leverages 35 years of expertise and glob…", 'RT @cyberappletech: Website Security Starts with Hosting\n\nCyber Apple keeps your site safe with:\n📌 DDoS Protection – Blocks attacks.\n📌 SSL…', 'RT @antgrasso: Prioritizing cybersecurity is not just a necessity but a responsibility, as overlooking fundamental protective measures can…', '🔐 A strong cybersecurity strategy isn’t just about responding to threats - it’s about preventing them.\n\nWe provide advanced vulnerability assessm

In [12]:
print('Number of tweets: ',len(tweet_data))

Number of tweets:  93


### Summarization 
- Analyze election tweets with delimiters 
- Change the size of the summarization 
- Summarize tweets and focus on different perspectives. 

In [13]:
messages = [
    {"role": "system", "content": f"""provide a brief summary of the tweets delimited by {delimiter}"""},
    {"role": "user", "content": f"{delimiter}{tweet_data}{delimiter}"},
    ]

print(openai_help(messages))

The tweets focus on the increasing importance of cybersecurity in today's digital landscape. They highlight the evolving nature of cyber threats, including data breaches, ransomware, and phishing attacks, and emphasize the need for proactive measures such as AI-driven security, vulnerability assessments, and robust data protection strategies. The role of AI in both enhancing cybersecurity and posing new threats is discussed, along with the importance of regular data backups and security compliance. Various organizations and events are mentioned, showcasing efforts to combat cyber threats and improve digital security across different sectors.


In [14]:
messages = [
    {"role": "system", "content": f"""provide a brief summary of the tweets delimited by {delimiter},
                                    limit the summary to 20 words"""},
    {"role": "user", "content": f"{delimiter}{tweet_data}{delimiter}"},
    ]

print(openai_help(messages))

The tweets emphasize the growing importance of cybersecurity, highlighting AI's role, proactive strategies, and the need for robust defenses against evolving cyber threats.


In [15]:
messages = [
    {"role": "system", "content": f"""provide a brief summary of the tweets delimited by {delimiter},
                                    focus on how people discuss AI,
                                    limit the summary to 50 words"""},
    {"role": "user", "content": f"{delimiter}{tweet_data}{delimiter}"},
    ]

print(openai_help(messages))

AI is discussed as a transformative force in cybersecurity, offering advanced threat detection and prevention capabilities. However, it also poses risks by enabling more sophisticated cyber threats. The conversation highlights AI's dual role as both a game-changer and a potential threat in the cybersecurity landscape.


### Moderation 
- Iterate each tweet and use the [moeration endpoint](https://platform.openai.com/docs/api-reference/moderations) to identify flagged tweets
- Print flagged tweets


In [16]:
def flag_help(tweet):
    response = client.moderations.create(
        model="omni-moderation-latest",
        input=tweet)

    if response.results[0].flagged:
        print('===')
        cat_dict = response.results[0].categories.to_dict()
        for cat in cat_dict.keys():
            if cat_dict.get(cat):
                print (cat)
                print(tweet)

In [17]:
for tweet in tweet_data:
    flag_help(tweet)

===
self-harm
RT @mbi_sponge: In this incident, in which a woman committed suicide, the following threats and cyber-lynching were made.
violence
RT @mbi_sponge: In this incident, in which a woman committed suicide, the following threats and cyber-lynching were made.


### Transforming
- Translating to a different language 
- Transform tones, such as formal vs. informal.  


In [18]:
for tweet in tweet_data:
    messages = [
        {"role": "system", "content": f"""translate the tweets delimited by {delimiter} into Chinese"""},
        {"role": "user", "content": f"{delimiter}{tweet}{delimiter} "}]

    print(openai_help(messages).strip(delimiter))

你的IT姿势如何？  
（提示：如果你的背挺直但防火墙薄弱……那我们有问题了。）

网络威胁不会懈怠，它们会袭击！

#RVA #RichmondIT https://t.co/1bfArg96gQ
网络安全

在当今的数字世界中，网络安全比以往任何时候都更加重要。随着技术的进步，网络威胁也在增加。从数据泄露到勒索软件攻击，保护敏感信息至关重要。
RT @TrendMicro: 趋势Cybertron：业界首个主动网络安全AI。我们的新AI代理利用了35年的专业知识和全球…
RT @cyberappletech: 网站安全始于托管

Cyber Apple 通过以下方式保护您的网站安全：
📌 DDoS 保护 – 阻止攻击。
📌 SSL…
RT @antgrasso: 优先考虑网络安全不仅是一种必要性，更是一种责任，因为忽视基本的保护措施可能会……
🔐 强大的网络安全策略不仅仅是应对威胁，还在于预防威胁。

我们提供先进的漏洞评估、AI驱动的威胁检测和虚拟安全服务，帮助企业领先于网络风险。

我们的最新视频… https://t.co/eY8sN7E1XE
人工智能在网络安全中的作用 随着网络威胁变得越来越复杂，传统的网络安全措施难以应对不断演变的攻击向量。人工智能（AI）在网络安全中已成为改变游戏规则的力量。https://t.co/yqqwm2SBHK
RT @antgrasso: 优先考虑网络安全不仅是必要的，而且是一种责任，因为忽视基本的保护措施可能会……
转发 @LindaGrass0: 网络威胁不再是“是否”发生的问题，而是“何时”发生的问题。您的组织准备好了吗？

一个积极主动的网络安全策略…
注意纽约、新泽西、加利福尼亚、佛罗里达、乔治亚或德克萨斯的汽车经销商！通过我们的全面汽车责任、车库和网络责任保险来保护您的业务和客户🔒 从事故到网络威胁，我们的风险保留团体保险为您提供全面保障🚙 #AutoDea
随着网络威胁变得越来越复杂，转向统一安全态势的方法至关重要。它为企业提供了安全成熟度的全面视图，帮助IT领导者跟踪进展、做出明智决策并优先考虑投资。 

https://t.co/krKYfaomlN https://t.co/1E6uxXUMw6
转发 @INTERPOL_HQ：如何在虚拟资产生态系统中打击金融犯罪和网络威胁？

根据……这是新的数字战

KeyboardInterrupt: 

In [19]:
for tweet in tweet_data:
    messages = [
        {"role": "system", "content": f"""rewrite the tweets delimited by {delimiter} in the tone like Stewie """},
        {"role": "user", "content": f"{delimiter}{tweet}{delimiter} "}]

    print(openai_help(messages).strip(delimiter))

Oh, do tell me about your IT posture, won't you? (Hint: If your spine is as straight as a ruler but your firewall is as flimsy as a wet noodle... well, we have a bit of a conundrum, don't we?) Cyber threats, much like myself, don't dawdle—they pounce! #RVA #RichmondIT https://t.co/1bfArg96gQ
Ah, the digital realm, where one's privacy is as fragile as a teacup in a bull's shop. In this age of technological marvels, cybersecurity is of utmost importance, wouldn't you agree? As our gadgets become more sophisticated, so too do the nefarious cyber scoundrels. From dastardly data breaches to those insidious ransomware attacks, safeguarding our precious information is simply non-negotiable.
RT @TrendMicro: Ah, behold Trend Cybertron, the industry's first proactive cybersecurity AI. Our new AI agent, a veritable marvel, draws upon 35 years of expertise and global prowess. Quite the impressive feat, wouldn't you say?
RT @cyberappletech: Ah, the delightful world of website security, where Cyber 

KeyboardInterrupt: 

### Inferring
- Use step-by-step instructions with delimiters to:
  1. Identify sentiments
  2. Identify emotions
  3. Extract mentioned people's names
  3. Identify whether a tweet supports Democratic, Republican, or unknown 
  4. Extract outputs into a structured JSON document. 
- Identify topics from Tweets. 


In [20]:
for tweet in tweet_data:
    messages = [
        {"role": "system", "content": f"""analyze the tweet delimited by {delimiter} in the following steps:
                                        step 1 {delimiter} identify the tweet sentiment in a single word, either positive, negative or neutral;
                                        step 2 {delimiter} identify the emotions expressed in the tweet with a single word;
                                        step 3 {delimiter} extract the mentioned peoples;
                                        step 4 {delimiter} detect whether the tweet support Democratic or Replublican, return the resunt in a single word;
                                        step 5 {delimiter} organize the result in a json document with the keys <sentiment>, <emontion>,<mentioned>, <support>
                                         Do not wrap the json codes in JSON markers and only return the json document"""},
        {"role": "user", "content": f"{delimiter}{tweet}{delimiter} "}]
    print(openai_help(messages))

{
  "sentiment": "neutral",
  "emotion": "caution",
  "mentioned": [],
  "support": ""
}
{
  "sentiment": "neutral",
  "emotion": "informative",
  "mentioned": [],
  "support": "neutral"
}
{
  "sentiment": "neutral",
  "emotion": "informative",
  "mentioned": ["TrendMicro"],
  "support": "neutral"
}
{
  "sentiment": "neutral",
  "emotion": "informative",
  "mentioned": ["cyberappletech"],
  "support": "neutral"
}
{
  "sentiment": "neutral",
  "emotion": "responsibility",
  "mentioned": ["antgrasso"],
  "support": "neutral"
}
{
  "sentiment": "neutral",
  "emotion": "informative",
  "mentioned": [],
  "support": "neutral"
}
{
  "sentiment": "neutral",
  "emotion": "informative",
  "mentioned": [],
  "support": "neutral"
}
{
  "sentiment": "neutral",
  "emotion": "responsibility",
  "mentioned": ["antgrasso"],
  "support": "neutral"
}
{
  "sentiment": "neutral",
  "emotion": "caution",
  "mentioned": ["LindaGrass0"],
  "support": "neutral"
}
{
  "sentiment": "neutral",
  "emotion": "info

In [21]:

messages = [
        {"role": "system", "content": f"""analyze the tweet delimited by {delimiter} to identify 10 topics, 
                                  Do not wrap the json codes in JSON markers """},
        {"role": "user", "content": f"{delimiter}{tweet_data}{delimiter} "}]
print(openai_help(messages))

{
  "Cybersecurity": "The importance of cybersecurity in protecting sensitive information and preventing cyber threats.",
  "AI in Cybersecurity": "The role of artificial intelligence in enhancing cybersecurity measures and its potential as a threat.",
  "Data Protection": "Strategies and practices for safeguarding data against cyber threats, including backups and recovery plans.",
  "Cyber Threats": "The evolving nature of cyber threats and the need for proactive measures to combat them.",
  "Ransomware": "Incidents of ransomware attacks and their impact on various sectors.",
  "Financial Crime": "The intersection of cyber threats and financial crime, particularly in the virtual asset ecosystem.",
  "Infrastructure Vulnerability": "The risk posed by cyber threats to critical infrastructure, including transportation and military mobilization.",
  "Identity Verification": "The importance of secure identity verification processes in the context of data privacy and cyber threats.",
  "Cyb

### Expanding with multiple prompts 
- Identify which party receives majority supports
- Provide contexts in the system message
- Create a chatbot to answer users’ inquiry  


In [22]:
analysis_result = []
from tqdm import tqdm
for tweet in tqdm(tweet_data):
    messages = [
        {"role": "system", "content": f"""analyze the tweet delimited by {delimiter} in the following steps:
                                        step 1 {delimiter} identify the tweet sentiment in a single word, either positive, negative or neutral;
                                        step 2 {delimiter} identify the emotions expressed in the tweet with a single word;
                                        step 3 {delimiter} extract the mentioned peoples;
                                        step 4 {delimiter} detect whether the tweet support Democratic or Replublican, return the resunt in a singple word;
                                        step 5 {delimiter} organize the result in a json document with the keys <sentiment>, <emontion>,<mentioned>, <support>
                                         Do not wrap the json codes in JSON markers and only return the json document"""},
        {"role": "user", "content": f"{delimiter}{tweet}{delimiter} "}]
    analysis_result.append(openai_help(messages))


100%|██████████| 93/93 [01:25<00:00,  1.09it/s]


In [ ]:
print(analysis_result)

In [23]:
messages = [
        {"role": "system", "content": f"""analyze the tweet analysis reuslt delimited by {delimiter} in the following steps:
                                        step 1 {delimiter} count the number of tweets that support Democratic and Republican;
                                        step 2 {delimiter} identify the common sentiments and emotoions to each mentioned people;
                                        step 3 {delimiter} organize the result in a json document with keys <Democratic count>, <Republican count>, <people name>
                                         Do not wrap the json codes in JSON markers and only return the json document"""},
        {"role": "user", "content": f"{delimiter}{analysis_result}{delimiter} "}]
analysis_summary = openai_help(messages)
print(analysis_summary)

{
  "Democratic count": 0,
  "Republican count": 2,
  "people name": {
    "Elon": {
      "sentiment": "neutral",
      "emotion": "anticipation"
    },
    "GovAbbottPress": {
      "sentiment": "neutral",
      "emotion": "concern"
    }
  }
}


## Create a chatbot

In [24]:
from openai import OpenAI

openai_api_key  = get_secret('openai')['api_key']
client = OpenAI(api_key=openai_api_key)
model = 'gpt-4o'
temperature = 0

chat_history = [

{"role": "system", "content": f"""you are a chabot answer user questions based on the tweets,
                                {delimiter}{tweet_data}{delimiter}, 
                                if user mentioned a people name in the {delimiter}{analysis_summary}{delimiter} people field,report the corresponding sentiment and emotion,
                            
                            """}
]

def chatbot(prompt):

    chat_history.append({"role": "user", "content": prompt})

    response = client.chat.completions.create(
        model=model,  # Use the model you prefer
        messages=chat_history
    )

    reply = response.choices[0].message.content

    chat_history.append({"role": "assistant", "content": reply})
    
    return reply

In [ ]:
while True:
    user_input = input("You: ")
    if user_input.lower() in ['exit', 'quit']:
        print("Chatbot: Goodbye!")
        break
    reply = chatbot(user_input)
    print(f"Chatbot: {reply}")

You:  what are the main topics mentioned?


Chatbot: The main topics mentioned in the tweets are cybersecurity, cyber threats, AI in cybersecurity, data protection, financial crime, ransomware attacks, digital security, network security, and backup strategies.


You:  what are people's feelings over the topic of cyber threats?


Chatbot: People's feelings over the topic of cyber threats, as inferred from the tweets, include:

1. Concern: There is a prevailing awareness and concern over the evolving and sophisticated nature of cyber threats.
2. Responsibility: Emphasis is placed on the responsibility to prioritize cybersecurity measures.
3. Urgency: There is a sense of urgency to protect against and stay ahead of cyber threats.
4. Anxiety: The persistent nature of threats can induce anxiety about potential data loss or breaches.
5. Anticipation: There is an anticipation for advanced solutions, such as AI, to mitigate these threats.

Overall, the sentiment leans towards being cautious and proactive in addressing cyber threats.


You:  what suggestions have they brought up to combat these issues?


Chatbot: The suggestions mentioned in the tweets to combat cyber threats include:

1. **Implementing Advanced Technologies**: Leveraging AI and machine learning to enhance threat detection and adapt to evolving attack vectors.

2. **Proactive Security Measures**: Developing robust cybersecurity strategies that focus not just on response, but also on preventing threats.

3. **Regular Data Backups**: Encouraging businesses and individuals to regularly back up data to prevent potential loss from cyber threats.

4. **Comprehensive Security Solutions**: Investing in multi-layered security systems that provide end-to-end protection for networks and data.

5. **Utilizing Secure Practices**: Adopting secure practices such as enabling two-factor authentication, using strong passwords, and avoiding public Wi-Fi.

6. **Cybersecurity Awareness**: Raising awareness about cybersecurity's importance and encouraging ongoing education to keep up with threats.

7. **Collaboration and Compliance**: Ensur

You:  end


Chatbot: If you have any more questions in the future or need further information, feel free to reach out. Stay safe and secure!


## Reference
- Isa Fulford and Andrew Ng. n.d.-a. *“Building Systems with the ChatGPT API.”* DeepLearning.AI. Accessed October 25, 2024. https://www.deeplearning.ai/short-courses/building-systems-with-chatgpt/.
- ———. n.d.-b. *“ChatGPT Prompt Engineering for Developers.”* DeepLearning.AI. Accessed October 25, 2024. https://www.deeplearning.ai/short-courses/chatgpt-prompt-engineering-for-developers/.
- OpenAI. n.d. *“OpenAI Documents.”* OpenAI. Accessed October 18, 2024. https://platform.openai.com.
